# Basic regression: Benzinpreis vorhersagen

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_absolute_error

## Dateset
### Daten herunterladen und in ein Dataframe speichern

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()

### Daten säubern

Zeilen mit leeren Felder löschen.

In [ ]:
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

Die `"Origin"` Spalte ist eigentlich categorical und nicht Numerisch. Dies müssen wir umbauen.


In [ ]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

### Aufteilen der Daten in Train und Test
Nun teilen wir den Datensatz in einen Trainingssatz und einen Testsatz auf.

Wir werden den Testsatz bei der Endauswertung unserer Modelle verwenden.

### Features von Labels trennen

Trennen Sie den Zielwert, das "Label", von den Features. Dieses Label ist der Wert, auf dessen Vorhersage Sie das Modell trainieren werden.

## Normalisierung

In der Statistiktabelle ist leicht zu erkennen, wie unterschiedlich die Bereiche der einzelnen Features sind.

Es ist eine gute Praxis, Features zu normalisieren, die unterschiedliche Skalierungen und Bereiche verwenden. 

Dies ist unter anderem deshalb wichtig, weil die Features mit den Gewichten der Modelle multipliziert werden. Die Skala der Ausgänge und die Skala der Gradienten werden also durch die Skala der Eingänge beeinflusst. 

Obwohl ein Modell ohne Feature-Normalisierung *möglicherweise* konvergiert, macht die Normalisierung das Training viel stabiler. 

### Der Normalization Layer
Der Layer `preprocessing.Normalization` ist eine saubere und einfache Möglichkeit, diese Vorverarbeitung in Ihr Modell einzubauen.

Der erste Schritt ist die Erstellung des Layers: 

Nun passen wir es den Daten mit `.adapt()` an:

Das berechnet für jedes Feature (Spalte) das mean und variance, welches dann im Layer gespeichert wird. 

## Linear regression

### Eine Variable


Das Training eines Modells mit `tf.keras` beginnt typischerweise mit der Definition der Modellarchitektur.

In diesem Fall verwenden Sie ein Sequentielles Modell. Dieses Modell stellt eine Abfolge von Schritten dar. In diesem Fall gibt es zwei Schritte:

* Normalisierung der eingegebenen `horsepower`.
* Wenden Sie eine lineare Transformation ($y = mx+b$) an, um 1 Ausgabe unter Verwendung von `layers.Dense` zu erzeugen.

Zuerst den horsepower `Normalization` layer:

### Model

Dieses Modell wird `MPG` aus `Horsepower` vorhersagen.

Lassen Sie das untrainierte Modell mit den ersten 10 Pferdestärkenwerten laufen. Das Ergebnis wird nicht gut sein, aber Sie werden sehen, dass es die erwartete Form `(10,1)` hat:

### Model trainieren

Sobald das Training konfiguriert ist, verwenden Sie Model.fit(), um das Training auszuführen:

### Evaluation - Wie ist das Training verlaufen?

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

Da es sich um eine Regression mit einer einzigen Variable handelt, ist es einfach, die Vorhersagen des Modells als Funktion des Inputs zu betrachten:

In [ ]:
def plot_horsepower(x, y):
  plt.scatter(train_features['Horsepower'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()